In [1]:
import pandas as pd
import numpy as np
import os
import sys
import re
import gc
import time
import warnings
warnings.filterwarnings("ignore")
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)
#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',100)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from collections import defaultdict 
from tqdm import tqdm

In [3]:
data_path = "./data/"
common_features_train_csv = os.path.join(data_path, "common_features_train.csv")
common_features_test_csv = os.path.join(data_path, "common_features_test.csv")

sample_skeleton_train_csv = os.path.join(data_path, "sample_skeleton_train.csv")
sample_skeleton_test_csv = os.path.join(data_path, "sample_skeleton_test.csv")

In [5]:
fea_cols = ['common_feature_index', 'feature_num2', 'feature_list2']
fea_train = pd.read_csv(common_features_train_csv, header=None, names=fea_cols, nrows=10000)
#sample_cols = ['sample_id', 'click', 'conversion', 'common_feature_index', 'feature_num1', 'feature_list1']
#sample_train = pd.read_csv(sample_skeleton_train_csv, nrows=100000, header=None, names=sample_cols)

In [17]:
sample_cols = ['sample_id', 'click', 'conversion', 'common_feature_index', 'feature_num1', 'feature_list1']
sample_train = pd.read_csv(sample_skeleton_train_csv, header=None, names=sample_cols, nrows=1000000)

In [5]:
sample_train.shape

(42300135, 6)

In [4]:
fea_train.shape
fea_train.head(2)

(730600, 3)

common_feature_index  feature_num2  \
0     84dceed2e3a667f8           343   
1     0000350f0c2121e7           811   

                                                                                         feature_list2  
0  101313191.012534387741.012634387791.012734387821.012838648851.012938648881.015...  
1  127_1437162241.94591127_1435146270.69315127_1437728710.69315127_1435432831.60944127_...

In [6]:
%%time
def feature_list_split(x):
    feature_dict = defaultdict(list)
    for fea in x.split('\x01'):
        field = re.split('\x02', fea)
        #fea = re.split('\x03', field[1])
        #feature_dict[field[0]].append({'feature_id':fea[0], 'value':fea[1]})
        feature_dict[field[0]].append(field[1])
    return feature_dict
fea_train['fea_dict2'] =  fea_train['feature_list2'].map(feature_list_split)

Wall time: 4.38 s


In [18]:
sample_train['fea_dict1'] =  sample_train['feature_list1'].map(feature_list_split)

In [9]:
sample_train['fea_dict1'][0].keys()

dict_keys(['210', '216', '301', '205', '206', '207', '508'])

In [20]:
sample_train['205'] = sample_train['fea_dict1'].map(lambda x:x['205']) # 商品ID
sample_train['206'] = sample_train['fea_dict1'].map(lambda x:x['206']) # 商品所属类目ID
sample_train['207'] = sample_train['fea_dict1'].map(lambda x:x['207']) # 商品所属店铺ID

sample_train['210'] = sample_train['fea_dict1'].map(lambda x:x['210']) # 商品关联用户意图ID：多值
sample_train['216'] = sample_train['fea_dict1'].map(lambda x:x['216']) # 商品的品牌ID
sample_train['508'] = sample_train['fea_dict1'].map(lambda x:x['508']) # 109_14和206域的组合特征：浮点值
sample_train['509'] = sample_train['fea_dict1'].map(lambda x:x['509']) # 110_14和207域的组合特征：浮点值
sample_train['702'] = sample_train['fea_dict1'].map(lambda x:x['702']) # 127_14和216域的组合特征：浮点值
sample_train['853'] = sample_train['fea_dict1'].map(lambda x:x['853']) # 150_14和210域的组合特征：多值，浮点值
sample_train['301'] = sample_train['fea_dict1'].map(lambda x:x['301']) # 业务场景信息的一种分类表示



In [18]:
sample_train[['205','206','702']].head()

,205,206,702
0,[41862221.0],[83167991.0],[]
1,[41868551.0],[83166761.0],[]
2,[44790031.0],[83165901.0],[98677624.02535]
3,[49339151.0],[83161771.0],[]
4,[49456631.0],[83165891.0],[]


In [25]:
# for i in ['205','206','207', '210','216','508','509', '702', '853', '301']:
#     print('-'*100 + i + '-'*100)
#     sample_train[i].map(lambda x: len(x)).value_counts()

for i in ['205','206','207', '210','216','509', '702', '853', '301']:
    print('-'*100 + i + '-'*100)
    sample_train[i].map(lambda x: x[0].split('\x03')[1] if len(x)>0 else 'a').value_counts()

----------------------------------------------------------------------------------------------------205----------------------------------------------------------------------------------------------------


1.0    999997
a           3
Name: 205, dtype: int64

----------------------------------------------------------------------------------------------------206----------------------------------------------------------------------------------------------------


1.0    999997
a           3
Name: 206, dtype: int64

----------------------------------------------------------------------------------------------------207----------------------------------------------------------------------------------------------------


...

a          741528
0.69315     59520
1.09861     27608
1.38629     17746
1.60944     14709
1.79176     11325
1.94591     10104
2.07944      7964
2.19722      7118
2.30259      5910
2.3979       5424
2.48491      4798
2.56495      4353
2.63906      4195
2.70805      3663
2.77259      3072
2.83321      3056
2.89037      2797
2.94444      2461
2.99573      2372
3.04452      2220
3.09104      1963
3.17805      1853
3.13549      1821
3.21888      1681
3.2581       1605
3.29584      1568
3.3322       1555
3.3673       1359
3.4012       1300
3.43399      1274
3.46574      1256
3.49651      1134
3.52636      1078
3.61092      1019
3.55535       978
3.68888       933
3.58352       923
3.63759       856
3.78419       814
3.66356       803
3.73767       769
3.71357       758
3.85015       729
3.7612        707
3.80666       693
3.8712        568
3.89182       564
3.82864       563
3.98898       558
3.91202       516
4.02535       512
3.95124       506
3.93183       501
4.12713       471
4.07754   

----------------------------------------------------------------------------------------------------853----------------------------------------------------------------------------------------------------


a          550470
0.60595     10486
0.34854      9341
0.98095      8536
0.51103      7767
0.28743      7040
0.81093      6974
0.69315      6869
1.25276      6675
1.09861      5908
0.84715      5728
1.46626      5300
0.40547      4748
1.79176      4165
0.91629      3975
0.77334      3720
1.29937      3661
1.38629      3639
1.20387      3625
1.1259       3567
0.55962      3470
1.64229      3259
1.60944      3256
0.15444      3217
0.45932      3115
1.55814      2958
0.22314      2864
1.36533      2740
1.0116       2723
0.07973      2612
2.14007      2542
1.04134      2463
1.50408      2445
2.03693      2396
1.73466      2360
1.94591      2338
0.65076      2319
1.92176      2236
1.15278      2178
1.17865      2112
0.73381      2098
1.70475      2088
1.54052      1977
0.88253      1955
0.94895      1869
1.84577      1851
1.34365      1780
2.3979       1774
1.67391      1749
2.07944      1743
2.19722      1726
1.07056      1698
1.32176      1691
1.4272       1684
1.8718       1654
1.981     

----------------------------------------------------------------------------------------------------301----------------------------------------------------------------------------------------------------


1.0    1000000
Name: 301, dtype: int64

In [22]:
sample_train[['508','509']].head()

,508,509
0,[93550390.69315],[]
1,[],[]
2,[93548375.14166],[95117692.30259]
3,[93544482.3979],[]
4,[93548363.29584],[]


In [16]:
sample_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 18 columns):
 #   Column                Non-Null Count    Dtype 
---  ------                --------------    ----- 
 0   sample_id             1000000 non-null  int64 
 1   click                 1000000 non-null  int64 
 2   conversion            1000000 non-null  int64 
 3   common_feature_index  1000000 non-null  object
 4   feature_num1          1000000 non-null  int64 
 5   feature_list1         1000000 non-null  object
 6   fea_dict1             1000000 non-null  object
 7   keys                  1000000 non-null  object
 8   205                   1000000 non-null  object
 9   206                   1000000 non-null  object
 10  207                   1000000 non-null  object
 11  210                   1000000 non-null  object
 12  216                   1000000 non-null  object
 13  508                   1000000 non-null  object
 14  509                   1000000 non-null  object
 15 

In [13]:
fea_train['fea_dict2'][0].keys()

dict_keys(['101', '125', '126', '127', '128', '129', '150_14', '121', '127_14', '109_14', '110_14', '122', '124'])

In [7]:
fea_train['101'] = fea_train['fea_dict2'].map(lambda x: x['101']) # 用户ID

fea_train['121'] = fea_train['fea_dict2'].map(lambda x: x['121']) # 用户的一种分类ID
fea_train['122'] = fea_train['fea_dict2'].map(lambda x: x['122']) # 用户的一种分类ID
fea_train['124'] = fea_train['fea_dict2'].map(lambda x: x['124']) # 用户性别分类ID

fea_train['125'] = fea_train['fea_dict2'].map(lambda x: x['125']) # 用户年龄分类ID
fea_train['126'] = fea_train['fea_dict2'].map(lambda x: x['126']) # 用户消费水平分类I
fea_train['127'] = fea_train['fea_dict2'].map(lambda x: x['127']) # 用户消费水平分类II
fea_train['128'] = fea_train['fea_dict2'].map(lambda x: x['128']) # 用户是否就业
fea_train['129'] = fea_train['fea_dict2'].map(lambda x: x['129']) # 用户地理信息分类ID

fea_train['150_14'] = fea_train['fea_dict2'].map(lambda x: x['150_14']) # 用户意图ID以及用户在该意图上的历史行为累积数量* ：多值
fea_train['127_14'] = fea_train['fea_dict2'].map(lambda x: x['127_14']) # 商品店铺ID以及用户在该店铺上的历史行为累积数量*
fea_train['109_14'] = fea_train['fea_dict2'].map(lambda x: x['109_14']) # 商品类目ID以及用户在该类目上的历史行为累积数量*
fea_train['110_14'] = fea_train['fea_dict2'].map(lambda x: x['110_14']) # 商品店铺ID以及用户在该店铺上的历史行为累积数量*



In [10]:
user_features = ['101','121','122','124','125','126','127', '128', '129', '150_14', '127_14', '109_14', '110_14']
item_features = ['205','206','207', '210','216','508','509', '702', '853', '301']
# for i in user_features:
#     print('-'*100 + i + '-'*100)
#     fea_train[i].map(lambda x: len(x)).value_counts()[0:5]

for i in user_features:
    print('-'*100 + i + '-'*100)
    fea_train[i].map(lambda x: x[0].split('\x03')[1] if len(x)>0 else 'a').value_counts()[0:5]

----------------------------------------------------------------------------------------------------101----------------------------------------------------------------------------------------------------


1.0    9968
a        32
Name: 101, dtype: int64

----------------------------------------------------------------------------------------------------121----------------------------------------------------------------------------------------------------


1.0    8659
a      1341
Name: 121, dtype: int64

----------------------------------------------------------------------------------------------------122----------------------------------------------------------------------------------------------------


...

0.69315    5397
1.09861    1365
1.38629     700
1.60944     467
1.79176     364
Name: 127_14, dtype: int64

----------------------------------------------------------------------------------------------------109_14----------------------------------------------------------------------------------------------------


0.69315    3371
1.09861    1278
1.38629     761
1.60944     517
1.79176     445
Name: 109_14, dtype: int64

----------------------------------------------------------------------------------------------------110_14----------------------------------------------------------------------------------------------------


0.69315    5663
1.09861    1346
1.38629     700
1.60944     415
1.79176     331
Name: 110_14, dtype: int64

In [11]:
for i in user_features:
    print('-'*100 + i + '-'*100)
    fea_train[i].map(lambda x: x[0].split('\x03')[0] if len(x)>0 else 'a').value_counts()[0:5]

----------------------------------------------------------------------------------------------------101----------------------------------------------------------------------------------------------------


a         32
123088     3
149470     3
342283     3
190955     2
Name: 101, dtype: int64

----------------------------------------------------------------------------------------------------121----------------------------------------------------------------------------------------------------


3438658    5114
a          1341
3438725     250
3438669     218
3438671     214
Name: 121, dtype: int64

----------------------------------------------------------------------------------------------------122----------------------------------------------------------------------------------------------------


...

3752845    154
3778829    124
a          118
3858194     72
3607774     67
Name: 127_14, dtype: int64

----------------------------------------------------------------------------------------------------109_14----------------------------------------------------------------------------------------------------


450988    488
450954    319
449089    266
451423    250
451267    248
Name: 109_14, dtype: int64

----------------------------------------------------------------------------------------------------110_14----------------------------------------------------------------------------------------------------


a          104
2826366     73
2430090     55
788270      45
1671251     37
Name: 110_14, dtype: int64

In [16]:
# 同一个用户
fea_train['user_id'] = fea_train['101'].map(lambda x: x[0].split('\x03')[0] if len(x)>0 else 'a')

In [18]:
fea_train[fea_train['user_id']=='123088'].head()

,common_feature_index,feature_num2,feature_list2,fea_dict2,101,121,122,124,125,126,127,128,129,150_14,127_14,109_14,110_14,user_id
7244,0ea7131a6b23d33e,380,150_1439476661.15278150_1439050600.81093150_1438684951.25276150_1439155980.55962150_...,"{'150_14': ['39476661.15278', '39050600.81093', '38684951.25276', '39155980.55962', '3867359...",[1230881.0],[34387211.0],[34387671.0],[34387681.0],[34387731.0],[34387781.0],[34387821.0],[38648851.0],[38648881.0],"[39476661.15278, 39050600.81093, 38684951.25276, 39155980.55962, 38673591.15278, 39519541....","[36624690.69315, 34527481.38629, 36350340.69315, 38566391.09861, 37785800.69315, 37923210....","[4535933.93183, 4549731.09861, 4509541.38629, 4517561.09861, 4508770.69315, 4512071.38629,...","[11199800.69315, 26523960.69315, 27700830.69315, 27825170.69315, 5933660.69315, 27508491.0...",123088
7603,0f6931b21582215a,380,127_1437766910.69315127_1436811530.69315127_1437451530.69315127_1434659251.09861127_...,"{'127_14': ['37766910.69315', '36811530.69315', '37451530.69315', '34659251.09861', '3601426...",[1230881.0],[34387211.0],[34387671.0],[34387681.0],[34387731.0],[34387781.0],[34387821.0],[38648851.0],[38648881.0],"[39568821.0116, 39045860.60595, 39631681.38629, 38983040.98095, 38993712.12022, 38767761.1...","[37766910.69315, 36811530.69315, 37451530.69315, 34659251.09861, 36014260.69315, 35165630....","[4535933.93183, 4549731.09861, 4509541.38629, 4517561.09861, 4508770.69315, 4512071.38629,...","[11199800.69315, 26523960.69315, 27700830.69315, 27825170.69315, 5933660.69315, 27508491.0...",123088
7750,0fbbe4e5d9e33fff,380,110_1417749950.69315110_1420325270.69315110_1418439891.38629110_1414667990.69315110_...,"{'110_14': ['17749950.69315', '20325270.69315', '18439891.38629', '14667990.69315', '1410184...",[1230881.0],[34387211.0],[34387671.0],[34387681.0],[34387731.0],[34387781.0],[34387821.0],[38648851.0],[38648881.0],"[38767761.1259, 39599921.71973, 38945620.60595, 39071750.60595, 38973183.02042, 39369212.8...","[35690540.69315, 38566391.09861, 38024190.69315, 37961850.69315, 37500951.09861, 38185392....","[4517561.09861, 4568120.69315, 4461941.09861, 4514831.09861, 4512052.07944, 4506583.63759,...","[17749950.69315, 20325270.69315, 18439891.38629, 14667990.69315, 14101840.69315, 28355680....",123088


In [19]:
fea_train['user_id'].nunique()

9830